In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import pystan
import pandas as pd
# add utilities directory to path
import os, sys
util_path = os.path.abspath(os.path.join(os.path.pardir, 'utilities_and_data'))
if util_path not in sys.path and os.path.exists(util_path):
    sys.path.insert(0, util_path)

# import from utilities
import stan_utility

In [2]:
df = pd.read_csv("kc_house_data.csv")
random_subset = df.sample(n = 3000)
random_subset = random_subset.values
print (random_subset.shape)
y = random_subset[:,2].ravel().astype(str).astype(float)
N = len(y)
x = random_subset[:,3:7].astype(str).astype(float)
F = 4
data = dict(
    N = N,
    F = F,
    y = y,
    x = x
)
print(x.shape)

(3000, 21)
(3000, 4)


In [3]:
model = stan_utility.compile_model('model.stan')

"""
stanCode = '''
data {  
  int <lower=0> N; // Number of data points. 
  int <lower=0> F; // Number of features
  real y[N]; // Price
  matrix[N,F] x; // Feature values
}
parameters {
  real intercept;
  vector[F] theta;
  real sigma;
}
model {
  vector[N] y_mean;
  y_mean <- intercept + x*theta;
  y ~ normal(y_mean, sigma);
}
'''
model = pystan.StanModel(model_code = stanCode)
"""

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f9ee4c66a3dc2a5cb7abcc785796e9f5 NOW.


In [4]:
fit = model.sampling(data = data, seed = 194838, n_jobs = -1)
samples = fit.extract(permuted = True)
print(fit)

Inference for Stan model: anon_model_f9ee4c66a3dc2a5cb7abcc785796e9f5.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

            mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
intercept  8.6e4  381.54  2.0e4  4.6e4  7.3e4  8.7e4 10.0e4  1.3e5   2740    1.0
theta[1]  -5.1e4  124.54 6513.5 -6.4e4 -5.5e4 -5.1e4 -4.6e4 -3.8e4   2735    1.0
theta[2]  3145.6  188.64 9167.5 -1.5e4  -2953 3111.1 9279.7  2.1e4   2362    1.0
theta[3]  300.02    0.17   8.34 283.77 294.69  299.9 305.59 316.25   2381    1.0
theta[4]   -0.36  1.7e-3    0.1  -0.57  -0.43  -0.36  -0.29  -0.16   3750    1.0
sigma      2.6e5   54.13 3355.8  2.5e5  2.6e5  2.6e5  2.6e5  2.6e5   3843    1.0
lp__      -3.9e4    0.04   1.77 -3.9e4 -3.9e4 -3.9e4 -3.9e4 -3.9e4   1857    1.0

Samples were drawn using NUTS at Thu Nov 22 17:45:11 2018.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale 